In [1]:
%matplotlib inline

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from bokeh.io import output_notebook, show
from bokeh.layouts import row, widgetbox
from bokeh.models import Select
from bokeh.palettes import Spectral5
from bokeh.plotting import curdoc, figure, ColumnDataSource
from bokeh.sampledata.autompg import autompg_clean as df

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score

ImportError: cannot import name 'autompg_clean'

In [2]:
vehicles = pd.read_csv('data/vehicles.csv', dtype=object)
weekly_gas_all_prices = pd.read_csv('data/GASALLCOVW.csv', index_col=0)
weekly_gas_reg_prices = pd.read_csv('data/GASREGCOVW.csv', index_col=0)
weekly_gas_prem_prices = pd.read_csv('data/GASPRMCOVW.csv', index_col=0)
annually_gas_all_prices = pd.read_csv('data/GASALLCOVA.csv', index_col=0)
annually_gas_reg_prices = pd.read_csv('data/GASREGCOVA.csv', index_col=0)
annually_gas_prem_prices = pd.read_csv('data/GASPRMCOVA.csv', index_col=0)

In [3]:
columns_for_vehicles = ['year',
                        'make',
                        'model',
                        'VClass',
                        'fuelType',
                        'fuelType1',
                        'city08',
                        'highway08',
                        'comb08']
vehicles = vehicles[columns_for_vehicles]
vehicles.head()

,year,make,model,VClass,fuelType,fuelType1,city08,highway08,comb08
0,1985,Alfa Romeo,Spider Veloce 2000,Two Seaters,Regular,Regular Gasoline,19,25,21
1,1985,Ferrari,Testarossa,Two Seaters,Regular,Regular Gasoline,9,14,11
2,1985,Dodge,Charger,Subcompact Cars,Regular,Regular Gasoline,23,33,27
3,1985,Dodge,B150/B250 Wagon 2WD,Vans,Regular,Regular Gasoline,10,12,11
4,1993,Subaru,Legacy AWD Turbo,Compact Cars,Premium,Premium Gasoline,17,23,19


In [4]:
categorical_columns = ['make', 'VClass', 'fuelType', 'fuelType1']
numeric_columns = ['city08', 'highway08', 'comb08']

In [23]:
for col in vehicles.columns:
    if col in categorical_columns:
        vehicles[col] = pd.Categorical(vehicles[col])
        vehicles['{col}_codes'.format(col=col)] = vehicles[col].cat.codes
    elif col in numeric_columns:
        vehicles[col] = pd.to_numeric(vehicles[col])


In [25]:
vehicles.head()
temp = vehicles[['make_codes', 'VClass_codes', 'fuelType_codes', 'comb08']]

In [ ]:
df = df.copy()

SIZES = list(range(6, 22, 3))
COLORS = Spectral5

# data cleanup
df.cyl = df.cyl.astype(str)
df.yr = df.yr.astype(str)
del df['name']

columns = sorted(df.columns)
discrete = [x for x in columns if df[x].dtype == object]
continuous = [x for x in columns if x not in discrete]
quantileable = [x for x in continuous if len(df[x].unique()) > 20]

def create_figure():
    xs = df[x.value].values
    ys = df[y.value].values
    x_title = x.value.title()
    y_title = y.value.title()

    kw = dict()
    if x.value in discrete:
        kw['x_range'] = sorted(set(xs))
    if y.value in discrete:
        kw['y_range'] = sorted(set(ys))
    kw['title'] = "%s vs %s" % (x_title, y_title)

    p = figure(plot_height=600, plot_width=800, tools='pan,box_zoom,reset', **kw)
    p.xaxis.axis_label = x_title
    p.yaxis.axis_label = y_title

    if x.value in discrete:
        p.xaxis.major_label_orientation = pd.np.pi / 4

    sz = 9
    if size.value != 'None':
        groups = pd.qcut(df[size.value].values, len(SIZES))
        sz = [SIZES[xx] for xx in groups.codes]

    c = "#31AADE"
    if color.value != 'None':
        groups = pd.qcut(df[color.value].values, len(COLORS))
        c = [COLORS[xx] for xx in groups.codes]
    p.circle(x=xs, y=ys, color=c, size=sz, line_color="white", alpha=0.6, hover_color='white', hover_alpha=0.5)

    return p


def update(attr, old, new):
    layout.children[1] = create_figure()


x = Select(title='X-Axis', value='mpg', options=columns)
x.on_change('value', update)

y = Select(title='Y-Axis', value='hp', options=columns)
y.on_change('value', update)

size = Select(title='Size', value='None', options=['None'] + quantileable)
size.on_change('value', update)

color = Select(title='Color', value='None', options=['None'] + quantileable)
color.on_change('value', update)

controls = widgetbox([x, y, color, size], width=200)
layout = row(controls, create_figure())

curdoc().add_root(layout)
curdoc().title = "Crossfilter"

In [ ]:
from bokeh.io import output_file, show

from bokeh.models import HoverTool
from bokeh.plotting import ColumnDataSource, figure
from bokeh.palettes import Spectral11

TOOLS = 'pan, wheel_zoom, box_zoom, reset, hover, save'

color_mapper = CategoricalColorMapper(factors=, 
                                      palette=Spectral11)

plot = figure(plot_width=900, plot_height=600, title="Vehicles Data", tools=TOOLS)

world_map_plot.patches(xs=lats,
                        ys=longs,
                        source=map_data,
                        fill_color="#F1EEF6",
                        fill_alpha=0.7,
                        line_width=2)

hover = world_map_plot.select_one(HoverTool)

hover.tooltips = [('Make', "@make"),
                ('Vehicle Class', '@VClass'),
                ('Fuel Type', '@fuelType1')
                ]

show(plot)